#### UFLP Model

In [3]:
using DataFrames
using XLSX
import XLSX
xf = XLSX.readxlsx("10node.xlsx")
XLSX.sheetnames(xf)
d = xf["Sheet1"]
location = d["A2:E12"]
location = DataFrame(location[2:end,:],location[1,:])
distances = d["A15:K25"]
distances = DataFrame(distances[2:end,2:end], :auto)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0.0,9.21954,3.0,7.07107,8.544,4.47214,6.7082,3.60555,5.09902,5.0
2,9.21954,0.0,7.61577,5.0,4.47214,5.0,3.16228,5.65685,6.08276,7.07107
3,3.0,7.61577,0.0,7.28011,5.83095,4.12311,6.0,3.16228,2.23607,2.0
4,7.07107,5.0,7.28011,0.0,8.06226,3.16228,2.23607,4.12311,7.2111,8.06226
5,8.544,4.47214,5.83095,8.06226,0.0,6.08276,5.83095,6.0,3.60555,4.24264
6,4.47214,5.0,4.12311,3.16228,6.08276,0.0,2.23607,1.0,4.24264,5.0
7,6.7082,3.16228,6.0,2.23607,5.83095,2.23607,0.0,3.16228,5.38516,6.32456
8,3.60555,5.65685,3.16228,4.12311,6.0,1.0,3.16228,0.0,3.60555,4.24264
9,5.09902,6.08276,2.23607,7.2111,3.60555,4.24264,5.38516,3.60555,0.0,1.0


In [10]:

function calc_trans_cost(location,distances,status,assignment) #Given the opening of the inventory and the assignment, calculate the cost.
    transp = 0
    for i in 1:length(status)
        if status[i] == false 
            transp += distances[i,assignment[i]]*location[i,"Demand"]*10
        else
            transp += 0
        end
    end
    return transp
end

function UFLP_heuristic(location,distances)
    status = falses(nrow(location)) #this is indicate whether a index is a inventory or not. If true it is an inventory, else it is a demand
    assignment = zeros(nrow(location)) #this is to indicate where are the demands assigned to.
    cost = typemax(Int64)#Initialize the cost to the largest possible number.
    temp_cost = typemax(Int64)#A temporary cost for each iteration, it will contain the minimum cost for each iteration.
    terminal = true #if all next possible opening will incur a larger cost we terminate.
    num = 0 #this represents the number of inventory open
    temp_final_assignment = zeros(Int,nrow(location)) #final assigment at the end of each iteration
    temp_assignment = zeros(Int,nrow(location)) #we need a temp_assigment for each possible iteration.
    while terminal
        for i in 1 : length(status) 
            if !status[i] #try openning locations at the place where there is no inventory
                status[i] = true
                #Now we need to decide the assigment of the demands
                #println("we set ", i , "to be a inventory: ", status)
                for j in 1:length(status)
                    if !status[j]
                        temp_assignment[j] = parse(Int,string(findmin(distances[j,status])[2])[2:end]) #assign the demand j to the inventory with the lowest distance
                    elseif status[j]
                        temp_assignment[j] = 0
                    end
                end
                if temp_cost >= sum(status.*location[1,"Fixed Cost"])+ calc_trans_cost(location,distances,status,temp_assignment)#this is to calculate the best, need to add in transportation cost
                    #need to update tempcost and the assigment
                    temp_cost = sum(status.*location[1,"Fixed Cost"])+ calc_trans_cost(location,distances,status,temp_assignment)
                    temp_final_assignment = copy(temp_assignment)
                    #println("our assigment is ", temp_final_assignment, "our cost is ",temp_cost)
                end
                status[i] = false
            end
        end
        if cost > temp_cost
            cost = temp_cost
            assignment = copy(temp_final_assignment)
            status[findall(x->x == 0,assignment)] .= true
            num = sum(status)
            println("we open ", num, " inventories in total, the assigments are given here ", assignment,"the cost is ",cost, "the transportation cost is ", calc_trans_cost(location,distances,status,assignment))
        else
            terminal = false
        end
    end
    println("we open inventory at: ",findall(x->x==true,status),"our assigment for the demand is listed here: ",assignment,"our cost is: ", cost)
end
UFLP_heuristic(location,distances)

we open 1 inventories in total, the assigments are given here [8, 8, 8, 8, 8, 8, 8, 0, 8, 8]the cost is 12850.823066732119the transportation cost is 12650.823066732119
we open 2 inventories in total, the assigments are given here [8, 8, 9, 8, 9, 8, 8, 0, 0, 9]the cost is 7878.5677119086085the transportation cost is 7478.5677119086085
we open 3 inventories in total, the assigments are given here [0, 8, 9, 8, 9, 8, 8, 0, 0, 9]the cost is 5915.236946630214the transportation cost is 5315.236946630214
we open 4 inventories in total, the assigments are given here [0, 7, 9, 7, 9, 8, 0, 0, 0, 9]the cost is 4413.4842767734635the transportation cost is 3613.4842767734635
we open 5 inventories in total, the assigments are given here [0, 7, 9, 7, 0, 8, 0, 0, 0, 9]the cost is 3423.652355870347the transportation cost is 2423.652355870347
we open 6 inventories in total, the assigments are given here [0, 0, 9, 7, 0, 8, 0, 0, 0, 9]the cost is 2769.8373876248843the transportation cost is 1569.8373876248

In [101]:
a=falses(10)
a[2] = a[7] = a[9] = true
a
#b = zeros(5)
#b .= typemax(Int64)
#findfirst(x -> x==minimum(distances[2,a]),a)
#sum(a.*2)
temp_assignment = zeros(nrow(location))
#temp_assignment
parse(Int,string(findmin(distances[4,a])[2])[2:end])


7